In [8]:
# Importing spark and dependencies

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [ ]:
# Pointing to our Google Project credentials

credentials_location = '/home/Adi/.gc/gcp_service_capstone.json'

In [ ]:
# Setting up spark cluster configurations

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [4]:
# Setting up spark cluster

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [38]:
# reading data as spark dataframes with previously 

df_appearances = spark.read\
                    .parquet('gs://capstone_datalake/raw/appearances.parquet')

df_competitions = spark.read\
                    .parquet('gs://capstone_datalake/raw/competitions.parquet')

In [49]:
# merging the data to form one dataset

df_data_all = df_appearances.join(df_competitions, on='competition_id', how='left')

In [ ]:
df_data_all.registerTempTable('data_all')

In [ ]:
spark.sql("""
SELECT *
FROM
    data_all
LIMIT 10
""").show()

In [53]:
df_data_all.coalesce(1).write.parquet('data_all.parquet', mode='overwrite')